In [ ]:
# Install dependencis
# Attribution: Geek for Geeks tutorial - https://www.geeksforgeeks.org/artificial-intelligence/introduction-to-langchain/

# -- CORE LANGCHAIN AND PLUGINS --
!pip install -U langchain langchain-openai langchain-community langchain-google-genai

# -- LARGE MODEL PROVIDERS/SKILL ADAPTERS --
!pip install -U google-generativeai huggingface_hub openai

# -- COMMON TOOLING AND UTILITIES --
!pip install -U python-dotenv yfinance duckduckgo-search

# -- DUCKDUCKGO SEARCH API WRAPPER --
!pip install -U ddgs

# -- OPTIONAL: For advanced memory (semantic search/vector db) --
!pip install -U faiss-cpu  # For in-memory vector DBs (Lightweight)
# If you want persistent/production memory, add chromadb or qdrant-client

# -- Install LangGraph for advanced agent orchestration --
!pip install -U langgraph

# -- (OPTIONAL) For running Python tool actions securely --
!pip install -U restrictedpython

# Restart the runtime after running this cell if prompted!

In [ ]:
# Import key libraries
import os
import requests
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_react_agent, AgentExecutor, initialize_agent, Tool, AgentType
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool  # newer import for @tool decorator
from langchain.memory import ConversationBufferMemory

In [ ]:
#MEMORY AGENT CELL 1
# ---- Session-scoped memory ----


from dataclasses import dataclass
from datetime import datetime
from typing import Dict, List, Optional, Any, Union
import re

@dataclass
class MemoryItem:
    symbol: str
    question: str
    answer: str
    created_at: str
    meta: Dict[str, Any]

class SessionMemory:
    def __init__(self, max_items: int = 200, max_per_symbol: int = 10):
        self._store: Dict[str, List[MemoryItem]] = {}
        self.max_items = max_items
        self.max_per_symbol = max_per_symbol

    def remember(self, symbol: str, question: str, answer: str, **meta) -> None:
        symbol = (symbol or "GENERIC").upper().strip()
        item = MemoryItem(
            symbol=symbol,
            question=(question or "").strip(),
            answer=(answer or "").strip(),
            created_at=datetime.utcnow().isoformat(timespec="seconds"),
            meta=meta or {}
        )
        bucket = self._store.setdefault(symbol, [])
        bucket.append(item)
        if len(bucket) > self.max_per_symbol:
            del bucket[0 : len(bucket) - self.max_per_symbol]
        self._cap_global()

    def recall(self, symbol: str, question: Optional[str] = None) -> Optional[str]:
        symbol = (symbol or "GENERIC").upper().strip()
        bucket = self._store.get(symbol, [])
        if not bucket:
            return None
        if not question:
            return bucket[-1].answer
        q = (question or "").strip()
        for item in reversed(bucket):
            if item.question == q:
                return item.answer
        return None

    def latest(self, symbol: str) -> Optional[MemoryItem]:
        symbol = (symbol or "GENERIC").upper().strip()
        bucket = self._store.get(symbol, [])
        return bucket[-1] if bucket else None

    def _cap_global(self):
        all_items = []
        for sym, bucket in self._store.items():
            for it in bucket:
                all_items.append((it.created_at, sym, it))
        if len(all_items) <= self.max_items:
            return
        all_items.sort(key=lambda x: x[0])  # oldest first
        to_drop = len(all_items) - self.max_items
        cutoff = set(id(it) for _, _, it in all_items[:to_drop])
        for sym in list(self._store.keys()):
            self._store[sym] = [it for it in self._store[sym] if id(it) not in cutoff]

SESSION_MEMORY = SessionMemory()

def extract_symbol(text: str) -> str:
    """
    Grab a likely ticker from the user_input like 'Analyze the SPY stock ticker'.
    Simple heuristic: first ALL-CAPS token 1-5 chars (e.g., AAPL, MSFT, SPY).
    Falls back to 'GENERIC' if none found.
    """
    if not text:
        return "GENERIC"
    candidates = re.findall(r"\b[A-Z]{1,5}\b", text)
    # Light filter for common English words
    stop = {"THE","AND","FOR","WITH","FROM","THIS","THAT","YOUR","HAVE","HOLD"}
    for c in candidates:
        if c not in stop:
            return c
    return "GENERIC"

def as_text(x: Any) -> str:
    """
    Normalize whatever comes back from planner/tools/evaluator/optimizer into a string.
    Works with LangChain AgentExecutor outputs (dict), AIMessage, or raw str.
    """
    try:
        # AIMessage / ChatMessage
        if hasattr(x, "content"):
            return str(x.content)
        # Agent-like dicts
        if isinstance(x, dict):
            if "output" in x and isinstance(x["output"], str):
                return x["output"]
            if "messages" in x and isinstance(x["messages"], list):
                return "\n\n".join(
                    (m.content if hasattr(m, "content") else str(m))
                    for m in x["messages"]
                )
        # plain string
        if isinstance(x, str):
            return x
        return str(x)
    except Exception:
        return str(x)


In [ ]:
#Set up LLM API Calls
from google.colab import userdata

gemini_key = userdata.get('GEMINI')
hf_key = userdata.get('HF_TOKEN')
openai_key = userdata.get('OPENAI')

In [ ]:
# Setup Gemini to use in Agents
from langchain_google_genai import ChatGoogleGenerativeAI
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=gemini_key
)

In [ ]:
# Test
# Test if Gemini LLM is reachable
try:
    response = llm_gemini.invoke("Say 'GoogleAPI-success-check'")
    print("Gemini API Response:", response)
except Exception as e:
    print("Error reaching Gemini API:", e)


In [ ]:
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(
    model="gpt-5-mini",  # or "gpt-3.5-turbo", "gpt-4-mini", etc.
    openai_api_key=openai_key,  # Your OpenAI API key
    temperature=0.0             # (optional) set as needed
)

In [ ]:
#Test
response = llm_openai.invoke("Explain how neural networks work in 10 words or less.")
print(response.content)

In [ ]:
'''
!pip install langchain-huggingface huggingface_hub

!pip install langchain_community

In [ ]:
'''
# For LangChain HuggingFace integration
from langchain_huggingface import HuggingFaceEndpoint

llm_hf = HuggingFaceEndpoint(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=hf_key   # Hugging Face API key
)

In [ ]:
'''
# Test
# Test if HF LLM is reachable
try:
    response = llm_hf.invoke("Describe a neural network in 10 words of less'")
    print("Gemini API Response:", response)
except Exception as e:
    print("Error reaching HuggingFace API:", e)

## Planning Agent
Plans research steps for stock analysis

0. Evaluate user input, infer that they want to know about a stock, or up to three stocks. They may or may not write the stock symbol.
1. Ask, what are the top things to know when analyzing any stock? (Shortcut by hitting top 3 analysis compaines? (Morningstar, Moody, Bloomberg?)
2. Research those things for a limited amount of tokens (how do we limit web search API counts? We could somehow hit an API like wikipedia or something similar if it exists)
3. Ingest news
4. Consider second order effects that might impact this stock or industry, serach news for that
5. Evaluate, compare and contrast and then summarize.
6. Tell tools agent which tools to use based on plan. Pass instructions of where to look and likely tools to use to Tools Agent. Also plan expected output,
- Stock Name
- Stock Symbol
- Value
- Category (growth or stock)
- Recommended strategy (buy, hold, sell)
- Brief pros and cons, including key organization news as well as indsutry trends.

In [ ]:
'''
#Potential Gemini Approach
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

simple_prompt = PromptTemplate.from_template(
    "You are a planning agent. Give a plan for: {input}"
)
chain = LLMChain(prompt=simple_prompt, llm=llm_gemini)

response = chain.invoke({"input": "Analyze the SPY stock ticker"})
print("Simple LLMChain response:", response)


In [ ]:
'''
# Gemini Approach
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Basic chain-of-thought agent, no true ReAct logic:
prompt = PromptTemplate.from_template(
    """
    You are an intelligent planner. For the following task, think step by step and devise a thorough plan:
    Task: {input}
    """)
chain = prompt | llm_gemini

result = chain.invoke({"input": "Analyze the SPY stock ticker"})
print(result)

In [ ]:
# ReAct Planning agent example from Geek for Geeks
# Define planning prompt
PLANNER_PROMPT = """
You are a thoughtful agent that reasons and acts in this format:

Thought: you should always think before acting
Action: create a step-by-step plan to analyze the stock
Action Input: the input to the action
Observation: the result of the action

(Repeat Thought/Action/Action Input/Observation as needed)
Thought: I now know the best plan
Final Answer: the final plan to provide to tool agent is

Question: {input}
{agent_scratchpad}
"""

@tool
def echo(text: str) -> str:
    """Return the input text as the output."""
    return text

tools = [echo]

# Create planner agent
planner_agent = create_react_agent(
    model=llm_openai,
    tools=tools,
    prompt=PLANNER_PROMPT
)


# --- Test input for the agent ---

#print("Sending to agent:", {"input": "Analyze the SPY stock ticker"})
#print(llm_gemini.invoke("test: does this call work?"))

response = planner_agent.invoke({"input": "Analyze the SPY stock ticker"})

# Helper to extract the response text (if needed)
try:
    output_text = response['output']    # Many agents return {'output': ...}
except (TypeError, KeyError):
    output_text = str(response)         # Fall back to direct string conversion

# You can use a tokenizer for exact token count,
# but most practical, use whitespace split for demonstration:
words = output_text.split()
first_100 = " ".join(words[:100])

print("First 100 words/tokens:\n")
print(first_100)

## Tool Agent
Integrates APIs (Yahoo Finance, SEC EDGAR, News APIs)

List of Tools:
- Web Search
- API call - Yahoo Finance

### Tool Functions


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Web search tool
search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="WebSearch",
    func=search.run,
    description="Search the web and return relevant results"
)

# Yahoo Finance tool
import yfinance as yf
def get_stock_price(ticker: str) -> str:
    stock = yf.Ticker(ticker)
    price = stock.info['regularMarketPrice']
    return f"{ticker} price is {price}"

yahoo_tool = Tool(
    name="YahooFinance",
    func=get_stock_price,
    description="Get the latest stock price for a given ticker symbol"
)

# OTHER APIs & TOOLS HERE IN THE FUTURE

# ]Python REPL tool
def python_eval(code: str) -> str:
    try:
        return str(eval(code))
    except Exception as e:
        return f"Error: {e}"

python_tool = Tool(
    name="PythonREPL",
    func=python_eval,
    description="Run Python code and return the result"
)

### Tools Agent Function

In [ ]:
# Tool execution agent

REACT_TOOL_PROMPT = """
You are an expert analyst and researcher. Using ONLY the tools provided, thoughtfully follow these steps:

1. Perform any pre-processing or data gathering needed.
2. Use all of the tools to answer or analyze the user's request. Answer qustions like "# Tool execution agent

You are an expert analyst and researcher. Using ONLY the tools provided, thoughtfully follow these steps:

1. Perform any pre-processing or data gathering needed.
2. Use all of the tools to answer or analyze the user's request and answer the following questions

* (“What is the current price of stock as of now?”

* “Search for news about stock published in the past 24 hours.”

“Calculate the latest annualized yield for stock using Yahoo data.”).
3. (If analysis requires multiple steps) Chain tools or aggregate intermediate results.
4. Summarize your findings in a concise and useful way for the end user.

At each stage, state your Thought, Action, Action Input, and Observation as needed.

Reminder: Do NOT make up information—use only direct tool outputs.

Final Answer: A succinct, well-organized summary of findings appropriate for the user.
Question: {input}
{agent_scratchpad}

"""

tools = [search_tool, yahoo_tool, python_tool]  # Pass tools above to pecialist list here

# Create tool

tools_executor = create_react_agent(
    model=llm_openai,
    tools=tools,
    prompt=REACT_TOOL_PROMPT
)

tools = [search_tool, yahoo_tool, python_tool]  # Pass tools above to pecialist list here

# Create tool

tools_executor = create_react_agent(
    model=llm_openai,
    tools=tools,
    prompt=REACT_TOOL_PROMPT
)

## Self Reflection & Evaluation Agent
Evaluates output quality and iterates

In [ ]:
# Define Self Evaluation agent

EVAL_PROMPT = """
You are an expert evaluator. Review the analysis/summary below for:

- Completeness (all important steps/points covered)
- Succinctness (concise, minimal repetition)
- Accuracy (supported by real or tool-sourced information)
- Clarity (well organized, easy to follow)

Give clear, specific suggestions if any improvement is needed.
Return the revised summary/answer if changes are warranted.
Otherwise, state that the answer is adequate.

--- Analysis To Evaluate ---
{input}
"""

self_evaluator = create_react_agent(
    model=llm_openai, # Or gemini_model if that's your variable
    tools = [],
    prompt=EVAL_PROMPT
)

## Optimization Agent

In [ ]:
# Define optimizer agent

OPTIMIZER_PROMPT = """
You are an optimization agent. Given the evaluator's feedback and the initial summary below,
produce a revised version that fixes any weaknesses cited (completeness, succinctness, accuracy, clarity).

Evaluator Feedback:
{feedback}
Initial Answer:
{answer}
---
Optimized Revised Answer:
"""

optimizer_agent = create_react_agent(
    model=llm_openai, # Or other model variable
    tools = [],
    prompt=OPTIMIZER_PROMPT
)

## Learning Agent
Maintains memory across analysis runs

In [ ]:
# ---- Learning Agent CELL 2 ----
# Maintains memory within the current Colab/kernel session only.

def maybe_answer_from_memory(user_input: str) -> Optional[str]:
    symbol = extract_symbol(user_input)
    # Try exact question match first
    ans = SESSION_MEMORY.recall(symbol, question=user_input)
    if ans:
        print(f"[Memory hit] Found exact match for {symbol}.")
        return ans
    # Fallback to latest for this symbol
    ans = SESSION_MEMORY.recall(symbol)
    if ans:
        print(f"[Memory hit] Using latest cached answer for {symbol}.")
        return ans
    return None

def remember_final_answer(user_input: str, final_answer: str, **meta):
    symbol = extract_symbol(user_input)
    SESSION_MEMORY.remember(
        symbol=symbol,
        question=user_input,
        answer=final_answer,
        **meta
    )
    print(f"[Memory write] Saved answer for {symbol}.")


## Multiple Agent Setup

In [ ]:
# UPDATED MAIN WORKFLOW WITH MEMORY AGENT

# Example user input
user_input = "Analyze the SPY stock ticker"

# 0) Try to answer directly from session memory
_cached = maybe_answer_from_memory(user_input)
if _cached:
    final_answer = _cached
    print("\n=== FINAL ANSWER (from session memory) ===\n")
    print(final_answer)
else:
    # 1) Planner
    print("Planner Agent Reasoning...")
    planner_output = planner_agent.invoke({"input": user_input})
    print("Planner Agent Output:", as_text(planner_output))
    print()

    # 2) Tools Executor
    print("Tools Executor Reasoning...")
    try:
        tools_output = tools_executor.invoke(
            {"input": as_text(planner_output)},
            config={"recursion_limit": 20}
        )
        print("Tools Executor Output:", as_text(tools_output))
    except Exception as e:
        tools_output = f"[Tools Exception] {type(e).__name__}: {e}"
        print("Tools Executor Exception:", type(e), e)
    print()

    # 3) Evaluator
    print("Evaluator Agent Reasoning...")
    evaluation_output = self_evaluator.invoke({"input": as_text(tools_output)})
    print("Evaluator Agent Output:", as_text(evaluation_output))
    print()

    # 4) Optimizer
    print("Optimizer Agent Reasoning...")
    # optimizer prompt expects==>> {feedback, answer}
    optimizer_input = {
        "feedback": as_text(evaluation_output),
        "answer": as_text(tools_output)
    }
    optimizer_output = optimizer_agent.invoke(optimizer_input)
    print("Optimizer Agent Output:", as_text(optimizer_output))
    print()

    # 5) Normalize final answer text
    final_answer = as_text(optimizer_output)

    
    def print_optimizer_output_text(text_block: str):
        print("Raw optimizer_output (normalized):\n")
        for para in str(text_block).split('\n\n'):
            print(para)
            print()

    print_optimizer_output_text(final_answer)

    # Remember the final answer in session memory
    remember_final_answer(
        user_input=user_input,
        final_answer=final_answer,
        planner_preview=as_text(planner_output)[:300],
        tools_preview=as_text(tools_output)[:300]
    )

    print("\n=/\=\/= FINAL ANSWER =\/=/\=\n")
    print(final_answer)
